In [5]:
!pip install torch transformers x-transformers betacal captum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.1 MB/s eta 0:00:00


In [2]:
# # Basic imports
import numpy as np  # For numerical computations and array manipulations
import pandas as pd  # For loading and handling time-series and static data
import sys
import importlib
import os
import time
from tqdm import tqdm
# PyTorch imports
import torch  # Core PyTorch library
import torch.nn as nn  # Neural network layers and loss functions
import torch.optim as optim  # Optimization algorithms
from torch.utils.data import Dataset, DataLoader  # Datasets and DataLoaders for batching
from torch.nn import Transformer, TransformerEncoderLayer  # Transformer modules
import re
# #Tranformers import
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import accuracy_score, precision_recall_curve, auc, roc_auc_score


module_path = '/home/workspace/files/MilanK/Model1/final_models/code'
# Add the module's directory to the system path if it's not already present
if module_path not in sys.path:
    sys.path.append(module_path)
    
    

from generating_datasets_for_torch import *
from load_static_data import *
from PatientDataset import *
from generate_labels_dataframe_with_dataloader import *
from load_train_test_split import *
from model import *
from load_patient_list import *
from forward_loop import *
from fit import *
from validate import *

In [3]:
train,val,test = load_train_test_data(
    train_filename = 'train_patient_list_orig.txt',                                   
    val_filename = 'val_patient_list_orig.txt',
    test_filename = 'test_patient_list.txt'
)



Training Set - Total Patients: 1214
Patients with Only Respiratory Deterioration: 7.50%
Patients with Only Cardiac Deterioration: 13.01%
Patients with Both Respiratory and Cardiac Deterioration: 5.93%
Patients with No Deterioration: 73.56%

Training Set - Total Windows: 8109
Windows with Only Respiratory Deterioration: 2.48%
Windows with Only Cardiac Deterioration: 4.06%
Windows with Both Respiratory and Cardiac Deterioration: 0.57%
Windows with No Deterioration: 92.90%

Validation Set - Total Patients: 150
Patients with Only Respiratory Deterioration: 4.67%
Patients with Only Cardiac Deterioration: 16.67%
Patients with Both Respiratory and Cardiac Deterioration: 6.00%
Patients with No Deterioration: 72.67%

Validation Set - Total Windows: 1022
Windows with Only Respiratory Deterioration: 2.25%
Windows with Only Cardiac Deterioration: 5.19%
Windows with Both Respiratory and Cardiac Deterioration: 0.59%
Windows with No Deterioration: 91.98%

Testing Set - Total Patients: 155
Patients wi

In [6]:
train_dataset = PatientDataset(patient_list = train, min_window_min=15, step_min=15,max_window_min=15,
                             prediction_window_length=15)


val_dataset = PatientDataset(patient_list = val, min_window_min=15, step_min=15,max_window_min=15,
                             prediction_window_length=15)


Generating 15-min samples: 100%|██████████| 1214/1214 [01:07<00:00, 17.87it/s]


Number of windows without minority oversampling: 8109


Generating 15-min samples: 100%|██████████| 150/150 [00:08<00:00, 18.07it/s]


Number of windows without minority oversampling: 1022


In [ ]:
batch_size = 128
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,             # Number of worker processes
    #prefetch_factor=1,          # Number of batches to preload per worker
    #persistent_workers=True      # Keep workers alive across epochs
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    #prefetch_factor=1,
    #persistent_workers=True
)




Generating 15-min samples: 100%|██████████| 155/155 [00:06<00:00, 22.45it/s]


Number of windows without minority oversampling: 1031


In [ ]:


model = run_exp(experiment_name = 'combined_model_simpler_demographics',
            train_loader = train_loader,
            val_loader = val_loader,
            depth=4,
            heads=2,
            dynamic_dim_in=18,
            dynamic_dim_out=128,
            max_seq_len=924,
            attn_dropout=0,
            ff_dropout=0,
            batch_norm=False,
            scalar_input_dim=40,
            scalar_mlp_hidden_dim=128,
            embedding_hidden_dim=128,
            ensemble_mlp_hidden_dim=512,
            output_dim=1,
            l1_lambda=0,
            l2_lambda=0,
            lr=3e-5,
            pos_weight_value=20.0,
            epochs = 5,
            max_norm=10,
            patience=10,
            lr_patience=5,
            device="cpu",
            target_outcome = 'cardiac')

Epoch 1/5
Epoch completed in 441.88s, Average Training Loss: 1.2449
Validation AUC-PR for resp: 0.1355
Validation AUROC for resp: 0.7732
Validation Loss: 1.1717
Validation AUPRC: 0.1355, Validation AUROC: 0.7732
Model weights saved for epoch 1
Epoch 1/5, Current Learning Rate: 0.000030
Epoch 2/5


/home/workspace/files/MilanK/Model1/final_models/code/fit.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metric_df = pd.concat([metric_df, epoch_df], ignore_index=True)


Epoch completed in 262.34s, Average Training Loss: 1.1258
Validation AUC-PR for resp: 0.1467
Validation AUROC for resp: 0.7879
Validation Loss: 1.0555
Validation AUPRC: 0.1467, Validation AUROC: 0.7879
Model weights saved for epoch 2
Epoch 2/5, Current Learning Rate: 0.000030
Epoch 3/5
Epoch completed in 262.33s, Average Training Loss: 1.0392
Validation AUC-PR for resp: 0.1654
Validation AUROC for resp: 0.7902
Validation Loss: 1.0109
Validation AUPRC: 0.1654, Validation AUROC: 0.7902
Model weights saved for epoch 3
Epoch 3/5, Current Learning Rate: 0.000030
Epoch 4/5
Epoch completed in 263.18s, Average Training Loss: 0.9926
Validation AUC-PR for resp: 0.1721
Validation AUROC for resp: 0.7908
Validation Loss: 0.9990
Validation AUPRC: 0.1721, Validation AUROC: 0.7908
Model weights saved for epoch 4
Epoch 4/5, Current Learning Rate: 0.000030
Epoch 5/5
Epoch completed in 260.75s, Average Training Loss: 0.9734
Validation AUC-PR for resp: 0.1693
Validation AUROC for resp: 0.7916
Validation L